In [12]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind, f_oneway

df = pd.read_csv("../data/interim/cleaned.csv" , low_memory=False)  

df["has_claim"] = (df["TotalClaims"] > 0).astype(int)
df["margin"] = df["TotalPremium"] - df["TotalClaims"]


In [14]:
# Contingency table of claim occurrence by province
import scipy.stats as stats

tbl = pd.crosstab(df["Province"], df["has_claim"])

chi2, p, dof, expected = stats.chi2_contingency(tbl)

print("Chi-square Province Risk Test:")
print("p-value =", p)


Chi-square Province Risk Test:
p-value = 1.0


In [19]:
df["PostalCode"] = df["PostalCode"].astype(str)

severity = df[df["has_claim"] == 1]

groups = [
    g["TotalClaims"].dropna().values
    for name, g in severity.groupby("PostalCode")
    if len(g["TotalClaims"].dropna()) >= 2
]

print("Number of groups for ANOVA:", len(groups))

if len(groups) >= 2:
    f_stat, p = stats.f_oneway(*groups)
    print("ANOVA Zipcode Severity Test: p =", p)
else:
    print("Not enough zipcode groups for ANOVA.")


Number of groups for ANOVA: 0
Not enough zipcode groups for ANOVA.


In [20]:
from scipy.stats import kruskal

if len(groups) >= 2:
    stat, p = kruskal(*groups)
    print("Kruskal-Wallis Zipcode Severity Test: p =", p)
else:
    print("Not enough data for any group-based test.")


Not enough data for any group-based test.


In [21]:
groups = [g["margin"].values for name, g in df.groupby("PostalCode")]

f_stat, p = stats.f_oneway(*groups)

print("Zipcode Margin Test p-value:", p)


Zipcode Margin Test p-value: 0.0


In [22]:
tbl = pd.crosstab(df["Gender"], df["has_claim"])
chi2, p, dof, exp = stats.chi2_contingency(tbl)
print("Gender Frequency Test p:", p)


Gender Frequency Test p: 1.0


In [23]:
m = severity[severity["Gender"]=="Male"]["TotalClaims"]
f = severity[severity["Gender"]=="Female"]["TotalClaims"]

t, p = stats.ttest_ind(m, f, equal_var=False)
print("Gender Severity T-test p:", p)


Gender Severity T-test p: nan


C:\Users\Admin\Desktop\kifiya-tasks\solar-challenge-week0\.venv\Lib\site-packages\scipy\_lib\deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)
